In [1]:
import os, re
import json
import pandas as pd

In [2]:
eng = './data/sbersquad_train_clean_final_translated.json'

rus = './data/sberquad/sbersquad_train_clean_final.json'


In [3]:
end_data = json.load(open(eng, 'r'))

In [4]:
rus_data = json.load(open(rus, 'r'))

In [9]:
rus_data['paragraphs'][0]

{'context': 'Вода, природный газ и электричество поставляются муниципалитетам кантона Женева государственной компанией Сервис-Андустриэль-ду-Женев (фр. Services Industriels de Genève). Большая часть питьевой воды (80 %) выкачивается из Женевского озера, а остальные 20 % из подземных вод, которые изначально поступают из реки Арв. 30 % электричества производится в самом кантоне тремя плотинами гидроэлектростанции на реке Рона: Сюже, Вербуа, Шанси-Пуни (фр. Seujet, Verbois, Chancy-Pougny). Кроме того, 13 % электричества производится в кантоне за счёт мусоросжигателя Ле-Шенувье (фр. Les Cheneviers). Остальные 57 % электричества поставляются из других кантонов Швейцарии и европейских стран. Государственная компания Сервис-Андустриэль-ду-Женев покупает электричество, которое произведено только за счёт методов возобновления энергии и никогда не использует электричество, произведённое на АЭС или ТЭС. Природный газ в город поставляет швейцарская компания Gaznat из Западной Европы, которая также

In [8]:
end_data['paragraphs'][0]

{'qas': [{'id': '6c590a8747403debe65d2fd031b27f2606700b88',
   'question_original': 'Где расположены плотины гидроэлектростанции, производящей 30 % электричества в Женеве?',
   'answers': [{'answer_start': -1, 'text': 'on the river Ron'}],
   'question': 'Where are the dams of a hydropower plant producing 30% of the electricity in Geneva?'},
  {'id': 'a437f03fd64d13d512e675d5026e6f3993bc9a82',
   'question_original': 'Какой вид электричества покупает государственная компания Сервис-Андустриэль-ду-Женев?',
   'answers': [{'answer_start': -1,
     'text': 'which is produced only by means of renewable energy'}],
   'question': 'What kind of electricity does the State-owned Service-Andustriel do-Jenev buy?'},
  {'id': '9b594816707d1034ef9f5aac82c6c148fb1d3bb4',
   'question_original': 'Сколько процентов электричества производится в кантоне за счёт мусоросжигателя Ле-Шенувье (фр. Les Cheneviers)?',
   'answers': [{'answer_start': -1, 'text': '13%'}],
   'question': 'How much electricity is 

In [11]:
len(end_data['paragraphs'])

8000

In [12]:
len(rus_data['paragraphs'])

8000

In [19]:
import random

In [15]:
from transformers import GPT2Tokenizer

In [18]:
tok = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3xl")
tok.add_special_tokens({"bos_token": "<s>"})
tok.add_special_tokens({"eos_token": "</s>"})


0

In [25]:
#eng 
def format_qa(eng_dic, rus_dic):
    lst = []
    for i in range(len(rus_dic['qas'])):
        example = tok.bos_token +"paragraph: "+eng_dic['context']+"\nlang: rus\nquestion: "+rus_dic['qas'][i]['question']+' answer: '+ rus_dic['qas'][i]['answers'][0]['text'] + tok.eos_token
        lst.append(example)
        example = tok.bos_token +"paragraph: "+rus_dic['context']+"\nlang: rus\nquestion: "+rus_dic['qas'][i]['question']+' answer: '+ rus_dic['qas'][i]['answers'][0]['text'] + tok.eos_token
        lst.append(example)
    for i in range(len(eng_dic['qas'])):
        example = tok.bos_token +"paragraph: "+eng_dic['context']+"\nlang: eng\nquestion: "+eng_dic['qas'][i]['question']+' answer: '+ eng_dic['qas'][i]['answers'][0]['text'] + tok.eos_token
        lst.append(example)
        example = tok.bos_token +"paragraph: "+rus_dic['context']+"\nlang: eng\nquestion: "+eng_dic['qas'][i]['question']+' answer: '+ eng_dic['qas'][i]['answers'][0]['text'] + tok.eos_token
        lst.append(example)
        

    return lst

In [26]:
cross_ling = []
for i in range(len(end_data['paragraphs'])):
    cross_ling += format_qa(end_data['paragraphs'][i], rus_data['paragraphs'][i])

In [27]:
len(cross_ling)

157388

In [28]:
cross_ling[0]

'<s>paragraph: Water, natural gas and electricity are supplied to the municipalities of the canton of Geneva by the State company Service-Andustriel do Genev (fr. Services Industries de Genève). Most of the drinking water (80 per cent) is pumped from the Lake Geneva and the remaining 20 per cent of the groundwater that originally comes from the River Arv. 30 per cent of the electricity is produced in the canton itself by three dams on the Ron River: Suje, Verbua, Shansi-Puni (fr. Seujet, Verbois, Chancy-Pougny). In addition, 13 per cent of the electricity is produced in the canton at the cost of the canon of the Les Chenoviers (f. Les Cheneviers). The remaining 57 per cent of the electricity is supplied from the other cantons of Switzerland and European countries. (The State company Servis-Andustriel-du-Jenev is buying electricity, which is produced only for the recovery of energy and is never used from the electric power plant, the power station, the power stations, the power stations

In [29]:
random.shuffle(cross_ling)
cross_ling[0]

'<s>paragraph: Вавилоно-ассирийская литература показывает различные стороны хозяйственной, общественной и повседневной жизни народов, населявших Древнюю Месопотамию в первые три тысячелетия до нашей эры. Отличительной чертой Вавилоно-ассирийской литературы является необычайный консерватизм всех литературных жанров. Задачей писца было не создание нового литературного произведения, а сохранение для будущих поколений литературного наследия древности или в лучшем случае применение древней литературы к потребностям и нуждам современности.\nlang: rus\nquestion: Какова была задача писца Вавилоно-ассирийской литературы? answer: сохранение для будущих поколений литературного наследия древности</s>'

In [30]:
batch_size = len(cross_ling) // 10

In [31]:
!mkdir /home/jovyan/data/crossqa

In [32]:
with open("/home/jovyan/data/crossqa/train.list", "w") as file:
    idx = 0
    while cross_ling:
        with open(f"/home/jovyan/data/crossqa/train{idx}.txt", "w") as file_t:
            for line in cross_ling[:batch_size]:
                file_t.write(f"{line}\n")
        file.write(f"/home/jovyan/data/crossqa/train{idx}.txt\n")
        idx += 1
        cross_ling = cross_ling[batch_size:]